In [7]:
import glob, re, nltk
from collections import Counter
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk import word_tokenize
from math import log
from math import sqrt
from nltk.corpus import stopwords
import numpy as np
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
word_doc = defaultdict(Counter)
doc_word = defaultdict(Counter)
vocab = set()
#read stop words
stop_words = set()
f = open('stops.txt', 'r')
for line in f:
    stop_words.add(line.strip())
f.close()
for w in stopwords.words('english'):
    stop_words.add(w)
print(len(stop_words), stop_words)

621 {'while', 'ok', 'tends', 'useful', "i'll", 'nd', "that's", "he's", "i'm", 'towards', 'yours', 'etc', 'got', 'viz', 'with', 'why', 'ex', "isn't", 'unlikely', 'she', 'gives', 'toward', 'relatively', '}', 'follows', 'non', 'selves', 'various', 'your', 'whereafter', 'next', "they've", 'co', 'theres', 'instead', 'welcome', '-', 'thereby', 'no', 'followed', 'have', 'here', 'former', 'almost', 'merely', 'never', 'apart', 'little', 't', 'use', "hadn't", 'try', 'outside', 'certainly', 'this', 'himself', 'an', "wasn't", 'along', 'available', 'know', 'own', 'okay', 'ask', 'anyhow', 'best', 'themselves', "we'd", 'nowhere', 'ie', 'other', 'when', 'containing', 'placed', '{', 'please', 'u', 'later', 'don', 'first', 'away', 'z', 'yes', 'whenever', 'want', 'either', 'help', "i've", 'twice', 'mainly', 'so', 'yet', 'corresponding', "c'mon", 'ourselves', 'else', 'somebody', 'uses', 'down', 'ought', 'done', 'brief', 'all', 'around', 'somewhere', 'eg', 'the', 'definitely', 'causes', 'be', 'didn', 'unti

In [9]:
def df(term):
    return len(word_doc[term])

def idf(term):
    return log(len(doc_word) / df(term), 10)

def prefixReplace(match):
    prefix, stem = match.group(1), match.group(2)
    temp = prefix + stem
    if not stem in stop_words:
        temp += ' ' + stem
    return temp

def hyphenReplace(match):
    temp = match.group()
    li = temp.split('-')
    temp = temp.replace('-', '')
    for item in li:
        if not item in stop_words:
            temp += ' ' + item
    return temp

def writeToFile(index_table):
    # write to output
    output_file = open('single_term_idx_table.txt', 'w')
    for key in sorted(index_table.keys()):
        output_file.write(' '.join(str(x) for x in key)+' '+str(index_table[key])+'\n')
    output_file.close()

def createIndexTable(input_path, doc_type):
    # format input_path to ./input_path/
    if input_path[0] == '/':
        input_path = '.' + input_path
    if input_path[-1] != '/':
        input_path += '/'
    wnl = WordNetLemmatizer()
    stemmer = PorterStemmer()
    files = glob.glob(input_path+'*') # grab all the files under path
    print(len(files))
    get_num = re.compile(r'(\d+)')
    p_prefix = re.compile(r'\b(a|an|ante|anti|auto|circum|co|com|con|contra|contro|de|dis|en|em|ex|extra|fore|hetero|homo|homeo|hyper|il|im|in|ir|inter|intra|intro|macro|micro|mid|mis|mono|non|omni|over|post|pre|pro|re|semi|sub|super|sym|syn|trans|tri|un|under|uni)-([a-z])+\b', re.I)
    p_hyphen = re.compile(r'\b(\w+-)+\w+\b')
    
    for file in files:
        f = open(file, 'r')
        docno = (doc_type,int(get_num.search(file).group(1)))
        for line in f:
            line = line.strip()
            line = line.lower()
            # expand stem with hyphen prefix
            line = p_prefix.sub(prefixReplace, line)
            # expand hyphenated word
            line = p_hyphen.sub(hyphenReplace, line)
            line = line.replace(':', ' ')
            line = line.replace('\'s', '')
            line = line.replace(',', ' ')
            line = line.replace('.', ' ')
            li = word_tokenize(line)
            li = [wnl.lemmatize(x) for x in li]
            li = [stemmer.stem(x) for x in li]
            for word in li:
                if not word in stop_words:
                    # build single term idx
                    word_doc[word][docno] += 1
                    doc_word[docno][word] += 1
                    vocab.add(word)

In [10]:
createIndexTable('../data/descriptions_train', 'train')
createIndexTable('../data/descriptions_test', 'test')
copy_vocab = vocab.copy()
for w in copy_vocab:
    if sum(word_doc[w].values()) <= 2:
        vocab.remove(w)
vocab_list = sorted(list(vocab))
print('num of vocab:', len(vocab_list))
train_d = []
test_d = []

for i in range(10000):
    tup = ('train', i)
    train_d.append([doc_word[tup][w] * idf(w) for w in vocab_list])
for i in range(2000):
    tup = ('test', i)
    test_d.append([doc_word[tup][w] * idf(w) for w in vocab_list])
np.save('bagOfWord_train_description_cutby2.npy', train_d)
np.save('bagOfWord_test_description_cutby2.npy', test_d)

print(len(train_d), len(train_d[0]))
print(len(test_d), len(test_d[0]))

print('finish description bag of word')
# writeToFile(single_term_idx)


10000
2000
num of vocab: 3455
10000 3455
2000 3455
finish description bag of word


In [11]:
print(test_d[0])
word_doc.clear()
doc_word.clear()
vocab.clear()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [12]:
createIndexTable('../data/tags_train', 'train')
createIndexTable('../data/tags_test', 'test')
copy_vocab = vocab.copy()
for w in copy_vocab:
    if sum(word_doc[w].values()) <= 2:
        vocab.remove(w)
vocab_list = sorted(list(vocab))
print('num of vocab:', len(vocab_list))
train_t = []
test_t = []
for i in range(10000):
    tup = ('train', i)
    train_t.append([doc_word[tup][w] * idf(w) for w in vocab_list])
for i in range(2000):
    tup = ('test', i)
    test_t.append([doc_word[tup][w] * idf(w) for w in vocab_list])
      
train_t = np.array(train_t)
test_t = np.array(test_t)
np.save('bagOfWord_train_tag_cutby2.npy', train_t)
np.save('bagOfWord_test_tag_cutby2.npy', test_t)

print(len(train_t), len(train_t[0]))
print(len(test_t), len(test_t[0]))
word_doc.clear()
doc_word.clear()
vocab.clear()
print('finish tag bag of word')

10000
2000
num of vocab: 102
10000 102
2000 102
finish tag bag of word


In [13]:
def empty_vec(data):
    for i in range(len(data)):
        if len(set(data[i])) == 1:
            print(i, end=' ')

print('\ntrain description empty vec')
empty_vec(train_d)
print('\ntest description empty vec')
empty_vec(test_d)
print('\ntrain tag vec')
empty_vec(train_t)
print('\ntest tag vec')
empty_vec(test_t)
print()


train description empty vec

test description empty vec

train tag vec
12 46 656 677 1312 1337 1364 1374 1410 1547 1615 1807 1831 2191 2253 2286 2375 2407 2772 2865 2911 3068 3327 3345 3388 3434 3437 3491 3554 3602 3668 3778 3861 3913 4089 4152 4166 4386 4417 4484 4633 4726 5054 5165 5191 5363 5513 5722 5805 5824 6094 6522 6678 6715 6776 6832 6966 7192 7318 7410 7849 7888 7973 8031 8080 8122 8329 8371 8447 8606 8649 8713 8732 8835 9103 9142 9305 9491 9566 9620 9713 9809 9911 9933 
test tag vec
452 607 763 784 860 1027 1030 1033 1059 1208 1340 1421 1437 1475 1574 1648 
